# Isodensity lines in R and $\rho$

In [ ]:
%matplotlib notebook

from IPython.core.display import display, HTML
from ipfnpytools.getsig import getsig
import ipfnpytools.aug_read as aug_read
from ipfnpytools.closest import closest
import matplotlib.pyplot as plt
from __future__ import print_function
from ipfnpytools.plot import plots
from warnings import warn
import numpy as np
from ipfnpytools.save_figure import save_figure
from ipfnpytools import rps_dump
import reflectometry as rfl
from scipy.signal import medfilt

## Parameters

Change at your own risk. Not guaranteed it will not explode

In [ ]:
# Shot number
shot = 37750

# Choose profile source between 'afs' and 'dump'
source = 'afs'

# If chosen 'dump', state the relative path
path = None

# Magnetic equillibrium
equillibrium = "EQH"

# Adjust Isodensity parameters
n_iso = 10           # Number of lines
start_density = 3e18 # First isodensity
stop_density = 3e19  # Last isodensity

# Median filter number of points (n=1 is equivalente to the unfiltered signal)
median_points = 11

In [ ]:
# Fetching data ---------------------------------------------------

time, lfs_signal, hfs_signal, lfs_area, hfs_area = rfl.fetch(shot, path if source=='dump' else None, verbose=True)

# Apply median filter to the radius
lfs_area = medfilt(lfs_area, (median_points, 1))
hfs_area = medfilt(hfs_area, (median_points, 1))

In [ ]:
# Preprocessing --------------------------------------------------

#Isodensities. Transpose `area` so density is the first index
lfs_isodensity = lfs_area.T
hfs_isodensity = hfs_area.T
density = lfs_signal[0,:]

print("lfs_isodensity", lfs_isodensity.shape)
print("density", density.shape)
print("time", time.shape)

last_index = closest(density, stop_density)
first_index = closest(density, start_density)

if (last_index - first_index + 1) < n_iso:
    warn("Impossible drawing " + str(n_iso) + " isodensities, drawing only " + str(last_index - first_index + 1))

# ------------------------ CHOOSE ONE -------------------------------    
    
# Guarantees uniform spacing but not inclusion of last isodensity
# drawing_indexes = range(first_index, last_index + 1, (last_index - first_index + 1)/n_iso)

# Guarantees inclusion of last isodensity but not uniform spacing
drawing_indexes = np.rint(np.linspace(first_index, last_index, n_iso)).astype(np.int)

print("Drawing_indexes:", drawing_indexes)

In [ ]:
fig, axes = plots(x=time, y=[lfs_isodensity[i] for i in drawing_indexes] + [hfs_isodensity[i] for i in drawing_indexes], 
                  boxes=[0]*n_iso + [1]*n_iso, 
                  labels=["%.2e" % density[i] for i in drawing_indexes] + [None]*len(drawing_indexes),
                  legend_kw={'loc':'center left', 'bbox_to_anchor':[1.02,-1,0.3,2]},
                  y_mins=[None]*n_iso + [None]*n_iso,
                  y_maxs=[None]*n_iso + [None]*n_iso,
                  sharex=True,
                  y_labels=["LFS\nR [m]", "HFS\nR [m]"],
                  title=str(shot))
fig.set_size_inches(10, 5)

# ------------ PARAMETERS ---------------
# axes[0].set_ylim(2.1, 2.25)  # LFS range in R
# axes[1].set_ylim(1.05, 1.10) # LFS range in R

# axes[0].set_xlim(2.0, 6.3) # Zoom in time

# axes[0].text(x=2.6, y=2, s="LFS", fontsize='x-large')
# axes[1].text(x=2.6, y=1.3, s="HFS", fontsize='x-large')
# plt.show()

In [ ]:
save_figure(fig, "%d_isodensity" % (shot))

## Adding separatrix information

In [ ]:
import dd #To read 
import kk_abock
import numpy as np
from ipfnpytools.trz_to_rhop import trz_to_rhop
from ipfnpytools.trhoptheta_to_rz import trhoptheta_to_rz
from ipfnpytools.getsig import getsig
import matplotlib.patheffects as pe

In [ ]:
# Set the antenna Z
zl = 0.14
zh = 0.07

# Get lower x point
rxpu = getsig(shot, 'GQH', 'Rxpu')
zxpu = getsig(shot, 'GQH', 'Zxpu')

# Get upper x point
rxpo = getsig(shot, 'GQH', 'Rxpo')
zxpo = getsig(shot, 'GQH', 'Zxpo')

rho_x_u = trz_to_rhop(rxpu.time, rxpu.data, zxpu.data, 
                      shot=shot, eq=equillibrium)
rho_x_u = np.diag(rho_x_u)

rho_x_o = trz_to_rhop(rxpo.time, rxpo.data, zxpo.data, 
                      shot=shot, eq=equillibrium)
rho_x_o = np.diag(rho_x_o)

print(rho_x_u)
print(rho_x_o)


# # Walls
# inner_wall = 1.045
# outer_wall = 2.22

# inner_wall_rho = trz_to_rhop(3, inner_wall, zh, shot, eq=equillibrium)
# outer_wall_rho = trz_to_rhop(3, outer_wall, zl, shot, eq=equillibrium)
# rhol = trz_to_rhop(time, lfs_area, np.ones_like(lfs_area)*zl, shot, eq=equillibrium)
# rhoh = trz_to_rhop(time, hfs_area, np.ones_like(hfs_area)*zh, shot, eq=equillibrium)

In [ ]:
# Discover R of secondary separatrix

r_second_hfs = []
for t, rho in zip(rxpo.time, rho_x_o):
    r_second_hfs.append(trhoptheta_to_rz(t, rho, np.pi, shot=shot, eq=equillibrium, verbose=False)[0])
r_second_hfs = np.array(r_second_hfs)
    
r_second_lfs = []
for t, rho in zip(rxpo.time, rho_x_o):
    r_second_lfs.append(trhoptheta_to_rz(t, rho, 0, shot=shot, eq=equillibrium, verbose=False)[0])  
r_second_lfs = np.array(r_second_lfs)

# Separatrix positions
rin = getsig(shot, 'GQH', 'Rin')
raus = getsig(shot, 'GQH', 'Raus')

**The next cell draws the separatrix and secondary separatrix on the previous plot**

To refresh the plot and erase the separatrix information simply re-execute the plotting cell 

In [ ]:
axes[0].plot(raus.time, raus.data, color='black', linestyle='--', label='sep.', alpha=0.6, path_effects=[pe.Stroke(linewidth=5, alpha=0.6, foreground='w'), pe.Normal()])
axes[1].plot(rin.time, rin.data, color='black', linestyle='--', label='sep.', alpha=0.6, path_effects=[pe.Stroke(linewidth=5, alpha=0.6, foreground='w'), pe.Normal()])

axes[0].plot(rxpo.time, r_second_lfs, color='red', linestyle='--', label='2$^{nd}$ sep.', alpha=0.6, path_effects=[pe.Stroke(linewidth=5, alpha=0.6, foreground='w'), pe.Normal()])
axes[1].plot(rxpo.time, r_second_hfs, color='red', linestyle='--', label='2$^{nd}$ sep.', alpha=0.6, path_effects=[pe.Stroke(linewidth=5, alpha=0.6, foreground='w'), pe.Normal()])

axes[0].legend(loc='center left', bbox_to_anchor=[1.02,-1,0.3,2]);

## Isodensities in $\rho$

In [ ]:
# Walls
inner_wall = 1.045
outer_wall = 2.22

inner_wall_rho = trz_to_rhop(time, inner_wall, zh, shot, eq=equillibrium)
outer_wall_rho = trz_to_rhop(time, outer_wall, zl, shot, eq=equillibrium)
rhol = trz_to_rhop(time, lfs_area, np.ones_like(lfs_area)*zl, shot, eq=equillibrium)
rhoh = trz_to_rhop(time, hfs_area, np.ones_like(hfs_area)*zh, shot, eq=equillibrium)

# Preprocessing --------------------------------------------------

#Isodensities. Transpose `area` so density is the first index
rhol_isodensity = rhol.T
rhoh_isodensity = rhoh.T
density = lfs_signal[0,:]

print("lfs_isodensity", lfs_isodensity.shape)
print("density", density.shape)
print("time", time.shape)

In [ ]:
fig2, axes2 = plots(x=time, y=[rhol_isodensity[i] for i in drawing_indexes] + [rhoh_isodensity[i] for i in drawing_indexes], 
                  boxes=[0]*n_iso + [1]*n_iso, 
                  labels=["%.2e" % density[i] for i in drawing_indexes] + [None]*len(drawing_indexes),
                  legend_kw={'loc':'center left', 'bbox_to_anchor':[1.02,-1,0.3,2]},
                  y_mins=[None]*n_iso + [None]*n_iso,
                  y_maxs=[None]*n_iso + [None]*n_iso,
                  sharex=True,
                  sharey=True,
                  y_labels=[r"$\rho$ (LFS)", r"$\rho$ (HFS)"],
                  title=str(shot))
fig2.set_size_inches(11, 4)
# plt.tight_layout()
# plt.xlim((2, 8))
plt.xlabel("Time [s]")

# axes[0].plot(raus.time, raus.data, color='black', linestyle='--', label='sep.', alpha=0.6, path_effects=[pe.Stroke(linewidth=5, alpha=0.6, foreground='w'), pe.Normal()])
# axes[1].plot(rin.time, rin.data, color='black', linestyle='--', label='sep.', alpha=0.6, path_effects=[pe.Stroke(linewidth=5, alpha=0.6, foreground='w'), pe.Normal()])

axes2[0].plot(rxpo.time, rho_x_o, color='red', linestyle='--', label='2$^{nd}$ sep.', alpha=0.6, path_effects=[pe.Stroke(linewidth=5, alpha=0.6, foreground='w'), pe.Normal()])
axes2[1].plot(rxpo.time, rho_x_o, color='red', linestyle='--', label='2$^{nd}$ sep.', alpha=0.6, path_effects=[pe.Stroke(linewidth=5, alpha=0.6, foreground='w'), pe.Normal()])

axes2[0].legend(loc='center left', bbox_to_anchor=[1.02,-1,0.3,2]);

plt.ylim(0.95, 1.15)